'제주도 은행' 카카오맵 크롤링

In [7]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [8]:
url = 'https://map.kakao.com'
driver = webdriver.Chrome()
driver.get(url)

# 지도 설정 이미지 클릭
img = driver.find_element(By.CLASS_NAME, 'inner_coach_layer')
img.click()

# 검색창에 '제주도 은행' 입력
search=driver.find_element(By.ID, 'search.keyword.query')
search.send_keys("제주도 은행")

driver.find_element(By.CLASS_NAME, 'go.ico_search.btn_search.active').click()

soup = bs(driver.page_source,"lxml")

In [9]:
# 은행 이름
title = soup.select("a.link_name")

for i in title:
    print(i.text)

# 은행 주소
addr = soup.select('.addr')

for i in addr:
    print(i.text)

# 은행 평점
score = soup.select(".rating.clickArea > .score")

for i in score:
    print(i.text)        

In [12]:
# 페이지 번호 수집
page = driver.find_elements(By.ID, "info.search.page > div > a")

for i in range(2, 6):
    driver.find_element(By.ID, f"info.search.page.no{i}").click()

전처리

In [ ]:
# 데이터 불러오기
df=pd.read_excel("./제주도 은행_크롤링 결과.xlsx")
df.head()

In [ ]:
# 불필요한 컬럼 삭제
df.drop(columns=['Column1', '평점'], inplace=True) # 원본에 그대로 적용

df['주소'] = df['주소'].str.replace('\n', ' ')

df.rename(columns={'이름':'은행명'}, inplace=True)

In [ ]:
df['은행명'].unique()

In [ ]:
# index를 포함하지 않고 데이터 저장
df.to_csv("제주도 은행지점 및 ATM 목록.csv", index=False)

In [ ]:
# 데이터 불러오기
df=pd.read_csv("./제주도 은행지점 및 ATM 목록.csv")
df.head()

In [ ]:
# 분할된 결과를 새로운 df에 저장
split_addr=df['주소'].str.split("(", expand=True) # 분할된 결과를 여러 열로 확장하여 반환

# 기존 df에 새로운 열로 추가
df['주소2'] = split_addr[0]

# 불필요한 컬럼 삭제
df.drop(columns=['주소'], inplace=True) # 원본에 그대로 적용

# 컬럼명 변경
df.rename(columns={'주소2':'주소'}, inplace=True)

In [ ]:
# ATM 관련 데이터 삭제

idx=df[df['은행명'].str.contains('365')].index
df=df.drop(idx)

idx=df[df['은행명'].str.contains('ATM')].index
df=df.drop(idx)

In [ ]:
df.to_csv("제주도 은행 목록.csv", index=False)

카카오 API 위도경도

In [ ]:
# API 키 가져오기
with open('C:/workspace/kakao.txt') as file:
    road_key = file.read()

# URL 만들기
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '제주특별자치도 제주시 남광북1길 1 제주법조타워 1,2층'
url = f'{base_url}?query={quote(addr)}'

# Header : Authorization: KakaoAK ${REST_API_KEY}
header = {'Authorization': f'KakaoAK {road_key}'}

# Kakao Local API 호출하여 결과 가져오기
result = requests.get(url, headers=header).json()
result

In [ ]:
lat = result['documents'][0]['y'] # 위도(latitude)
lng = result['documents'][0]['x'] # 경도(latitude)
dong=result['documents'][0]['address']['region_3depth_name']

In [ ]:
# 제주도 은행 정보 완성하기
df = pd.read_csv("./제주도 은행 목록.csv")

In [ ]:
lat_list, lng_list, dong_list = [], [], [] # 빈 리스트 생성. 위도와 경도 값 저장
for i in df.index: # df의 각 행에 대해 반복
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json() # API에 요청을 보내고 응답을 JSON 형식으로 받음
    try:
        lat_list.append(result['documents'][0]['y']) # 위도(y) 추출하여 lat_list에 추가
        lng_list.append(result['documents'][0]['x']) # 경도(x) 추출하여 lng_list에 추가
        dong_list.append(result['documents'][0]['address']['region_3depth_name'])
    except:
        print(df.은행명[i])  
# try-except 구문: try블록 실행 중 예외 발생 시 except블록 실행

In [ ]:
df[df.은행명.isin(['호남새마을금고 도남지점'])].주소

In [ ]:
#카카오맵에서 직접 검색해서 주소 수정
df.iloc[97]['주소'] = '제주특별자치도 제주시 청사로 85'
df.iloc[97]

In [ ]:
df[df.은행명.isin(['제주은행 제주특별자치도지점'])].주소

In [ ]:
#카카오맵에서 직접 검색해서 주소 수정
df.iloc[122]['주소'] = '제주특별자치도 제주시 문연로 6'
df.iloc[122]

In [ ]:
# 수정된 데이터로 다시 위도경도 받아오기
lat_list, lng_list, dong_list = [], [], [] # 빈 리스트 생성. 위도와 경도 값 저장
for i in df.index: # df의 각 행에 대해 반복
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json() # API에 요청을 보내고 응답을 JSON 형식으로 받음
    try:
        lat_list.append(result['documents'][0]['y']) # 위도(y) 추출하여 lat_list에 추가
        lng_list.append(result['documents'][0]['x']) # 경도(x) 추출하여 lng_list에 추가
        dong_list.append(result['documents'][0]['address']['region_3depth_name'])
    except:
        print(df.은행명[i])
# try-except 구문: try블록 실행 중 예외 발생 시 except블록 실행

In [ ]:
df['위도']=lat_list
df['경도']=lng_list
df['주소2']=dong_list

# 제주도 기념품샵 위도 경도 동 리스트
lat_list
lng_list
dong_list

In [ ]:
df.to_csv("제주도 은행 목록.csv", index=False)

지도 시각화

In [ ]:
pd.read_csv('./제주도 은행 목록.csv')

In [ ]:
import folium
import pandas as pd

# folium.Map() 함수 사용하여 지도 객체 생성하기
# df의 위도와 경도 열의 평균값인 위치에 해당하는 지도를 생성하고 확대 수준을 10으로 설정
map=folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=10)
map

In [ ]:
# 제주시(blue)-서귀포시(green) 구분해서 folium.Marker
tiles = "http://mt0.google.com/vt/lyrs=m&hl=ko&x={x}&y={y}&z={z}" # 구글지도 타일 설정
attr= "Google" # 속성 설정
map = folium.Map(location=[df['위도'].mean(), df['경도'].mean()], # folium을 사용하여 지도 초기화
                 zoom_start=10, tiles=tiles, attr=attr)
for i in df.index: # df의 각 행을 반복
    if '제주시' in df.loc[i, '주소']:
        folium.Marker(
            location=[df.loc[i, '위도'], df.loc[i, '경도']],
            tooltip=df.loc[i, '은행명'], # 마커에 마우스를 올렸을 때 표시되는 툴팁으로 병원명 표시
            popup=folium.Popup(df.loc[i, '주소'], max_width=200),
            icon=folium.Icon(color='green', icon='glyphicon-usd')
        ).add_to(map)
    else:
        folium.Marker(
            location=[df.loc[i, '위도'], df.loc[i, '경도']],
            tooltip=df.loc[i, '은행명'], # 마커에 마우스를 올렸을 때 표시되는 툴팁으로 병원명 표시
            popup=folium.Popup(df.loc[i, '주소'], max_width=200),
            icon=folium.Icon(color='blue', icon='glyphicon-usd')
        ).add_to(map)

map        